In [1]:
import ee
import geemap as emap

In [19]:
ee.Initialize()
ee.Authenticate()

Enter verification code: 4/1AX4XfWh6cCsoLx9mZDQiuG69zEtrNEPNJ_N1Z2TyBsIvknDgBA-yhPLtX_Q

Successfully saved authorization token.


# Basemap 

In [2]:
Map=emap.Map(center=[21,105], zoom=7)
Map

Map(center=[21, 105], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(To…

In [3]:
# Show all basemap availables
Map.basemap_demo()
Map

Map(center=[21, 105], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(To…

# Split Map

In [44]:
# Understand the US land cover map
lcmap=ee.ImageCollection("USGS/NLCD")

lcmap.size().getInfo() # There are 14 land cover maps in the collection

14

In [45]:
# See specific land covermaps 
lcmap.aggregate_array("system:id").getInfo()

['USGS/NLCD/NLCD1992',
 'USGS/NLCD/NLCD2001',
 'USGS/NLCD/NLCD2001_AK',
 'USGS/NLCD/NLCD2001_HI',
 'USGS/NLCD/NLCD2001_PR',
 'USGS/NLCD/NLCD2004',
 'USGS/NLCD/NLCD2006',
 'USGS/NLCD/NLCD2008',
 'USGS/NLCD/NLCD2011',
 'USGS/NLCD/NLCD2011_AK',
 'USGS/NLCD/NLCD2011_HI',
 'USGS/NLCD/NLCD2011_PR',
 'USGS/NLCD/NLCD2013',
 'USGS/NLCD/NLCD2016']

In [55]:
nlcd_2001 = ee.Image('USGS/NLCD/NLCD2001').select('landcover') # US land cover map 2001
nlcd_2016 = ee.Image('USGS/NLCD/NLCD2016').select('landcover') # US land cover map 2016
# Define the zoom map
Map=emap.Map()

left_layer = emap.ee_tile_layer(nlcd_2001, {}, 'NLCD 2001') # Create left_layer map 
right_layer = emap.ee_tile_layer(nlcd_2016, {}, 'NLCD 2016') # Create right_layer map

Map.split_map(left_layer, right_layer)
Map

Map(center=[40, -100], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

# Drawing and Feature Collection

In [80]:
# Get US Map
usmap=ee.FeatureCollection("TIGER/2018/States")
Map.addLayer(usmap,{},"USMAP")
Map

Map(bottom=1994.0, center=[29.6880527498568, -94.74609375000001], controls=(WidgetControl(options=['position',…

In [81]:
# See drawing feature objects
Map.draw_features # See how many drawing done or Map.draw_last_features => print the last drawing

In [82]:
# Convert the drawing to featurecollection or Feature
drawing=ee.FeatureCollection(Map.draw_features)

In [83]:
# Convert one feature
poly=ee.Feature(Map.draw_features[2]) # point feature

In [84]:
Map.addLayer(poly,{},"Poly")
Map

Map(bottom=1994.0, center=[29.6880527498568, -94.74609375000001], controls=(WidgetControl(options=['position',…

# Interactive Map

In [104]:
import json
import requests
import os
from geemap import geojson_to_ee, ee_to_geojson 
from ipyleaflet import GeoJSON, Marker, MarkerCluster

In [106]:
file_path = r"F:\DLR_PhD\GEE\Youtube_Tutorial\us-cities.json"
if not os.path.exists(file_path):
    url = 'https://github.com/giswqs/geemap/raw/master/examples/data/us-cities.json'
    r = requests.get(url)
    with open(file_path, 'w') as f:
        f.write(r.content.decode("utf-8"))        

with open(file_path) as f:
    json_data = json.load(f)

In [115]:
marker_cluster=MarkerCluster(markers=[Marker(location=feature["geometry"]["coordinates"][::-1]) for feature in json_data["features"]])

In [116]:
Map.add_layer(marker_cluster)
Map

Map(bottom=1994.0, center=[29.6880527498568, -94.74609375000001], controls=(WidgetControl(options=['position',…

# Shapefile to GEE object and vice versa

In [122]:
VN_ee=emap.shp_to_ee(r"F:\Research\Maps\Maps\VN_Map\vnm_admbnda_adm1_gov_20200103.shp")

In [125]:
emap.ee_to_shp(VN_ee,filename=r"F:\DLR_PhD\GEE\Youtube_Tutorial\VN.shp")

Request payload size exceeds the limit: 10485760 bytes.


In [ ]:
# We can export ee to csv
emap.ee_export_vector(VN_ee,filename=r"F:\DLR_PhD\GEE\Youtube_Tutorial\VN.csv") # we can also convert it to "VN.kml", "VN.json"